In [ ]:
# import libraries
import pandas as pd
import numpy as np
import cv2
from numpy import asarray
from PIL import Image
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam 
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
%matplotlib inline

# access the root to the dataset
from google.colab import drive
drive.mount('/content/drive')

# read the CSV with all the dataset
df = pd.read_csv("dataset.csv")

# store Data in Compressed NumPy array files (.NPZs)
npz_paths = []

for i, row in df.iterrows():

    picture_path = row['Path']

    npz_path = picture_path.split('.')[0] + '.npz'
    npz_paths.append(npz_path)

    pic_bgr_arr = cv2.imread(picture_path)

    pic_image_size = cv2.resize(pic_bgr_arr, (96, 96))

    image = Image.fromarray(pic_image_size)

    norm_image = asarray(image)

    norm_image = norm_image.astype('float32')

    norm_image /= 255.0
  
    pic_rgb_arr = cv2.cvtColor(norm_image, cv2.COLOR_BGR2RGB)

    age = row['Age']
 

    stats = np.array([age])

    diagnosis = row['Dx']
    np.savez_compressed(npz_path, pic=pic_rgb_arr, stats=stats,   diagnosis=  diagnosis)

df['NPZ_Path'] = pd.Series(npz_paths)

# get dataframe of Age Only

stats_df = df[['Age']]

# calculate the mean of the column

means = [stats_df[col].mean() for col in stats_df]

# calculate the std. deviation of the column

std_devs = [stats_df[col].std() for col in stats_df]

# create tensorFlow preprocessing function for stats stream

def stat_scaler(tensor):
  return (tensor - means) / std_devs

# reduce to how many columns you feed
stat_scaler(tf.constant([2], dtype=tf.float32))

# shuffle the dataframe, and split into train, validation and test sets
shuffled_df = df.sample(frac =1)

# train_df, val_df, test_df = shuffled_df[:int], shuffled_df[int:int], shuffled_df[int:]
train_df, val_df = shuffled_df[:int], shuffled_df[int:int]
test_df = test_df_1[int:]

# create function to get X_pic, X_stats, and y from a dataFrame

def get_X_y(df):

  X_pic, X_stats = [], []
  y = []

  for name in df['NPZ_Path']:
    loaded_npz = np.load(name)

    pic = loaded_npz['pic']
    X_pic.append(pic)

    stats = loaded_npz['stats']
    X_stats.append(stats)
    
    y.append(loaded_npz['diagnosis'])

  X_pic, X_stats = np.array(X_pic), np.array(X_stats)
  y = np.array(y)

  return (X_pic, X_stats), y

(X_train_pic, X_train_stats), y_train = get_X_y(train_df)


# get the training data

(X_train_pic.shape, X_train_stats.shape), y_train.shape

# get the valalidation data

(X_val_pic, X_val_stats), y_val = get_X_y(val_df)

(X_val_pic.shape, X_val_stats.shape), y_val.shape

# get the test data

(X_test_pic, X_test_stats), y_test = get_X_y(test_df)

(X_test_pic.shape, X_test_stats.shape), y_test.shape

# define the model
# define the picture (CNN) stream

input_pic = layers.Input(shape=(96, 96, 3))
x         = layers.Lambda(preprocess_input)(input_pic)
x         = MobileNet(input_shape=(96, 96, 3), dropout=0.2,
                      include_top=False)(x)
x         = layers.GlobalAveragePooling2D()(x)
x         = layers.Dense(32, activation="relu")(x)
x         = layers.Dense(64, activation='relu')(x)
x         = layers.Dense(128, activation='relu')(x)
x         = Model(inputs=input_pic, outputs=x)

# define the stats (Age) Stream
input_stats = layers.Input(shape=(1,))
y = layers.Lambda(stat_scaler)(input_stats)
y = layers.Dense(32, activation="relu")(y)
y = layers.Dense(64, activation="relu")(y)
y = layers.Dense(128, activation="relu")(y)
y = layers.Dense(10, activation="relu")(y)
y = Model(inputs=input_stats, outputs=y)

# concatenate the two streams together
combined = layers.concatenate([x.output, y.output])

# define joined (FNN) Layer
z = layers.Dense(4, activation="relu")(combined)

# define output node of 1 linear neuron (regression task)
z = layers.Dense(1, activation="sigmoid")(z)


# define the final model
model = Model(inputs=[x.input, y.input], outputs=z)

# compile the model with Adam optimizer and mean-squared-error loss function
model.compile(loss='mse',
              optimizer=Adam(learning_rate=0.01),
              metrics= ['accuracy'])
              
# create a model saving callback and train for epochs connected to GPU runtime!!

cp = ModelCheckpoint('model_name', save_best_only=True)

hist = model.fit(
        x= [X_train_pic, X_train_stats], 
        y= y_train, 
        validation_data=([X_val_pic, X_val_stats], y_val),
        epochs=20,
        batch_size = 19, 
        callbacks=[cp])
        
# Load the saved model  

loaded_model = load_model('model_name')

# use the loaded model to obtain predictions with qEEG images and age in test set

test_predictions = loaded_model.predict([X_test_pic, X_test_stats]).flatten()

# add predictions column to the dataset to compare

test_df.reset_index(drop=True, inplace=True)

test_df['Predicted Diagnosis'] = test_predictions

test_df.to_csv("Predicted.csv")

# Convert predictions to a pandas series for easy prediction and plot a histogram
test_preds_series = pd.Series(test_predictions)

test_preds_series.hist()